In [32]:
import os
import json
import math
import pickle
import joblib
from typing import List, Dict, Optional, Tuple
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from sklearn.decomposition import TruncatedSVD
import networkx as nx
from sklearn.preprocessing import normalize

In [33]:
try:
    import implicit
    HAS_IMPLICIT = True
except Exception:
    HAS_IMPLICIT = False

In [34]:
def ensure_genres_list(g):
    # Handle iterable types first to avoid ambiguous truth value in pd.isna
    if isinstance(g, list):
        return g
    if isinstance(g, (tuple, set)):
        return list(g)
    if isinstance(g, np.ndarray):
        return list(g.tolist())

    # Now handle missing/NaN
    if g is None:
        return []
    if isinstance(g, float) and pd.isna(g):
        return []

    # Parse strings like "a|b" or "a, b"
    if isinstance(g, str):
        if '|' in g:
            return [x.strip() for x in g.split('|') if x.strip()]
        return [x.strip() for x in g.split(',') if x.strip()]

    # Best-effort fallback
    try:
        return list(g)
    except Exception:
        return []

In [35]:
def build_mappings(interactions: pd.DataFrame, items: pd.DataFrame):
    users = interactions['user_id'].unique().tolist()
    items_list = items['item_id'].unique().tolist()
    user2idx = {u: i for i, u in enumerate(users)}
    idx2user = {i: u for u, i in user2idx.items()}
    item2idx = {it: i for i, it in enumerate(items_list)}
    idx2item = {i: it for it, i in item2idx.items()}
    return user2idx, idx2user, item2idx, idx2item


def build_user_item_matrix(interactions: pd.DataFrame, user2idx, item2idx, implicit=True):
    n_users = len(user2idx)
    n_items = len(item2idx)
    mat = np.zeros((n_users, n_items), dtype=float)
    for _, row in interactions.iterrows():
        u = row['user_id']
        it = row['item_id']
        if u not in user2idx or it not in item2idx:
            continue
        ui = user2idx[u]
        ii = item2idx[it]
        if 'rating' in row and not implicit:
            val = float(row['rating']) if not pd.isna(row['rating']) else 0.0
        else:
            val = 1.0
        mat[ui, ii] += val
    return mat

In [36]:
def precision_at_k(recommended: List[str], ground_truth: set, k: int) -> float:
    if k == 0:
        return 0.0
    recommended_k = recommended[:k]
    if not recommended_k:
        return 0.0
    hits = sum(1 for r in recommended_k if r in ground_truth)
    return hits / k


def recall_at_k(recommended: List[str], ground_truth: set, k: int) -> float:
    if not ground_truth:
        return 0.0
    recommended_k = recommended[:k]
    hits = sum(1 for r in recommended_k if r in ground_truth)
    return hits / len(ground_truth)


def dcg_at_k(recommended: List[str], ground_truth: set, k: int) -> float:
    dcg = 0.0
    for i, r in enumerate(recommended[:k]):
        rel = 1.0 if r in ground_truth else 0.0
        denom = math.log2(i + 2)
        dcg += rel / denom
    return dcg


def ndcg_at_k(recommended: List[str], ground_truth: set, k: int) -> float:
    idcg = 0.0
    # ideal DCG: put all ground truth items at top
    ideal_hits = min(len(ground_truth), k)
    for i in range(ideal_hits):
        idcg += 1.0 / math.log2(i + 2)
    if idcg == 0:
        return 0.0
    return dcg_at_k(recommended, ground_truth, k) / idcg

In [37]:
def train_test_split_leave_one_out(interactions: pd.DataFrame, by_time_col: Optional[str]=None, seed: int=42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    For each user, leave one interaction for test. If `by_time_col` provided, choose the latest interaction.
    Otherwise choose a random interaction as test for each user with >=2 interactions.
    Users with only 1 interaction will have test empty (or can be handled separately).
    """
    rng = np.random.default_rng(seed)
    train_rows = []
    test_rows = []
    grouped = interactions.groupby('user_id')
    for user, group in grouped:
        if by_time_col and by_time_col in group.columns:
            group_sorted = group.sort_values(by=by_time_col)
            test_idx = group_sorted.index[-1]
        else:
            if len(group) == 1:
                # keep in train to avoid empty training
                test_idx = None
            else:
                test_idx = rng.choice(group.index)
        for idx, row in group.iterrows():
            if idx == test_idx:
                test_rows.append(row.to_dict())
            else:
                train_rows.append(row.to_dict())
    train_df = pd.DataFrame(train_rows)
    test_df = pd.DataFrame(test_rows)
    return train_df, test_df

In [38]:
class PopularityRecommender:
    def __init__(self):
        self.pop_scores = None
        self.item_order = None

    def fit(self, interactions: pd.DataFrame, item2idx: Dict[str,int]):
        # compute popularity by item counts in interactions
        counts = interactions['item_id'].value_counts()
        # map to item list order
        self.pop_scores = counts.to_dict()
        # fallback to items not present
        self.item_order = [it for it, _ in counts.items()]

    def recommend(self, user_id: str, seen: set, top_k: int=20):
        recs = []
        for it in self.item_order:
            if it in seen:
                continue
            recs.append(it)
            if len(recs) >= top_k:
                break
        return recs

In [39]:
class SVDRecommender:
    def __init__(self, n_factors: int=50, random_state: int=42):
        self.n_factors = n_factors
        self.random_state = random_state
        self.user_factors = None
        self.item_factors = None
        self.svd = None

    def fit(self, train_mat: np.ndarray):
        n_components = min(self.n_factors, min(train_mat.shape)-1)
        svd = TruncatedSVD(n_components=n_components, random_state=self.random_state)
        U = svd.fit_transform(train_mat)
        Vt = svd.components_
        self.user_factors = U
        self.item_factors = Vt.T
        self.svd = svd

    def recommend(self, user_idx: int, seen_idx: set, idx2item: Dict[int,str], top_k: int=20):
        if self.user_factors is None or self.item_factors is None:
            return []
        scores = self.user_factors[user_idx].dot(self.item_factors.T)
        candidates = []
        for iid, sc in enumerate(scores):
            if iid in seen_idx:
                continue
            candidates.append((iid, sc))
        candidates.sort(key=lambda x: x[1], reverse=True)
        return [idx2item[iid] for iid, _ in candidates[:top_k]]

In [40]:
class ALSRecommender:
    def __init__(self, factors=50, regularization=0.01, iterations=20):
        if not HAS_IMPLICIT:
            raise RuntimeError('implicit library not available; install with `pip install implicit`')
        self.model = implicit.als.AlternatingLeastSquares(factors=factors, regularization=regularization, iterations=iterations)
        self.factors = factors

    def fit(self, train_mat: np.ndarray, user2idx, item2idx):
        # implicit expects item-user sparse matrix
        from scipy.sparse import coo_matrix
        item_user = coo_matrix(train_mat.T)
        # train
        self.model.fit(item_user)
        # item factors are model.item_factors

    def recommend(self, user_id, user2idx, idx2item, N=20):
        uid = user2idx[user_id]
        recs = self.model.recommend(uid, None, N=N)
        # returns list of (item_idx, score)
        return [idx2item[i] for i, _ in recs]

In [41]:
class GraphRecommender:
    def __init__(self):
        self.G = None

    def build_graph(self, interactions: pd.DataFrame):
        G = nx.Graph()
        # add nodes with prefixes
        for u in interactions['user_id'].unique():
            G.add_node(f'u_{u}', bipartite=0)
        for it in interactions['item_id'].unique():
            G.add_node(f'i_{it}', bipartite=1)
        for _, r in interactions.iterrows():
            G.add_edge(f"u_{r['user_id']}", f"i_{r['item_id']}")

        self.G = G

    def recommend(self, user_id: str, seen: set, top_k: int=20, alpha: float=0.85):
        if self.G is None:
            return []
        start = f'u_{user_id}'
        if start not in self.G:
            return []
        pr = nx.pagerank(self.G, alpha=alpha, personalization={start: 1.0})
        items_scores = []
        for node, sc in pr.items():
            if node.startswith('i_'):
                it = node[2:]
                if it in seen:
                    continue
                items_scores.append((it, sc))
        items_scores.sort(key=lambda x: x[1], reverse=True)
        return [it for it, _ in items_scores[:top_k]]

In [42]:
def find_similar_by_genre_author(item_id: str, items_df: pd.DataFrame, item2idx: Dict[str,int], pop_scores: Dict[str,float], top_k: int=10):
    if item_id not in items_df['item_id'].values:
        return []
    row = items_df[items_df['item_id']==item_id].iloc[0]
    genres = set(row['genres'])
    author = row.get('author', None)
    candidates = []
    for _, r in items_df.iterrows():
        if r['item_id'] == item_id:
            continue
        score = len(set(r['genres']) & genres)
        if author and r.get('author')==author:
            score += 1
        pop = pop_scores.get(r['item_id'], 0.0)
        candidates.append((r['item_id'], score, pop))
    candidates.sort(key=lambda x: (x[1], x[2]), reverse=True)
    return [c[0] for c in candidates[:top_k]]


def cold_start_for_user(user_id: str, items_df: pd.DataFrame, user_train_interactions: pd.DataFrame, pop_scores: Dict[str,float], top_k: int=20):
    user_items = user_train_interactions[user_train_interactions['user_id']==user_id]['item_id'].tolist()
    if not user_items:
        # global popular
        ordered = sorted(pop_scores.items(), key=lambda x: x[1], reverse=True)
        return [it for it, _ in ordered[:top_k]]
    # infer genres
    genres = Counter()
    for it in user_items:
        gs = items_df[items_df['item_id']==it]['genres'].iloc[0]
        for g in gs:
            genres[g]+=1
    top_genres = set([g for g,_ in genres.most_common(3)])
    candidates = []
    for _, r in items_df.iterrows():
        overlap = len(set(r['genres']) & top_genres)
        pop = pop_scores.get(r['item_id'], 0.0)
        score = overlap + 0.2*pop
        candidates.append((r['item_id'], score))
    candidates.sort(key=lambda x: x[1], reverse=True)
    return [c[0] for c in candidates[:top_k]]

In [43]:
def evaluate_models(train_interactions: pd.DataFrame, test_interactions: pd.DataFrame, items: pd.DataFrame, models: Dict[str, object], k_list: List[int]=[5,10,20], save_dir: str='models') -> pd.DataFrame:
    """
    models: dict with keys as model names and values as objects exposing .fit and .recommend or custom callables.
    For SVD/ALS/Popularity/Graph we adhere to the interfaces above.
    Returns DataFrame with metrics per model per k.
    """
    os.makedirs(save_dir, exist_ok=True)
    # prepare mappings and matrices
    user2idx, idx2user, item2idx, idx2item = build_mappings(train_interactions, items)
    train_mat = build_user_item_matrix(train_interactions, user2idx, item2idx)
    # popularity scores
    pop_counts = train_interactions['item_id'].value_counts().to_dict()

    results = []

    # fit all models
    for name, model in models.items():
        print('Fitting', name)
        if name == 'popularity':
            model.fit(train_interactions, item2idx)
        elif name == 'svd':
            model.fit(train_mat)
        elif name == 'als':
            if HAS_IMPLICIT:
                model.fit(train_mat, user2idx, item2idx)
            else:
                print('Skipping ALS (implicit not installed)')
                continue
        elif name == 'graph':
            model.build_graph(train_interactions)
        # save model artifacts
        model_path = os.path.join(save_dir, f'{name}.pkl')
        try:
            joblib.dump(model, model_path)
        except Exception as e:
            print('Could not save model via joblib:', e)
        # if graph, also save graph separately
        if name == 'graph' and hasattr(model, 'G') and model.G is not None:
            gp_path = os.path.join(save_dir, 'graph.gpickle')
            try:
                with open(gp_path, 'wb') as f:
                    pickle.dump(model.G, f)
            except Exception as e:
                print('Could not pickle graph:', e)

    # evaluate per user in test set
    test_grouped = test_interactions.groupby('user_id')
    users_eval = list(test_grouped.groups.keys())
    # We'll restrict to users present in train mappings
    users_eval = [u for u in users_eval if u in user2idx]

    for name, model in models.items():
        print('Evaluating', name)
        # accumulate per-k metrics
        metrics_acc = {k: {'precision':[], 'recall':[], 'ndcg':[]} for k in k_list}
        for user in users_eval:
            uidx = user2idx[user]
            # seen items (train)
            seen_idx = set(np.where(train_mat[uidx] > 0)[0])
            seen = set([idx2item[i] for i in seen_idx])
            # ground truth (test items for user)
            ground_truth = set(test_grouped.get_group(user)['item_id'].tolist())
            if not ground_truth:
                continue
            # get recommendations (model-specific)
            if name == 'popularity':
                recs = model.recommend(user, seen, top_k=max(k_list))
            elif name == 'svd':
                recs = model.recommend(uidx, seen_idx, idx2item, top_k=max(k_list))
            elif name == 'als':
                if not HAS_IMPLICIT:
                    continue
                recs = model.recommend(user, user2idx, idx2item, N=max(k_list))
            elif name == 'graph':
                recs = model.recommend(user, seen, top_k=max(k_list))
            else:
                # custom callable
                try:
                    recs = model(user, top_k=max(k_list))
                except Exception:
                    recs = []
            for k in k_list:
                p = precision_at_k(recs, ground_truth, k)
                r = recall_at_k(recs, ground_truth, k)
                n = ndcg_at_k(recs, ground_truth, k)
                metrics_acc[k]['precision'].append(p)
                metrics_acc[k]['recall'].append(r)
                metrics_acc[k]['ndcg'].append(n)
        # aggregate
        for k in k_list:
            if len(metrics_acc[k]['precision'])==0:
                avg_p = avg_r = avg_n = 0.0
            else:
                avg_p = float(np.mean(metrics_acc[k]['precision']))
                avg_r = float(np.mean(metrics_acc[k]['recall']))
                avg_n = float(np.mean(metrics_acc[k]['ndcg']))
            results.append({'model':name, 'k':k, 'precision':avg_p, 'recall':avg_r, 'ndcg':avg_n})
    res_df = pd.DataFrame(results)
    return res_df

In [44]:
if __name__ == '__main__':
    # small synthetic example
    items = pd.DataFrame([
        {'item_id':'b1','title':'Book A','genres':['Fantasy','Adventure'],'author':'Auth1'},
        {'item_id':'b2','title':'Book B','genres':['Sci-Fi'],'author':'Auth2'},
        {'item_id':'b3','title':'Book C','genres':['Fantasy'],'author':'Auth3'},
        {'item_id':'b4','title':'Book D','genres':['History'],'author':'Auth2'},
        {'item_id':'b5','title':'Book E','genres':['Sci-Fi','Adventure'],'author':'Auth1'}
    ])
    interactions = pd.DataFrame([
        {'user_id':'u1','item_id':'b1'},
        {'user_id':'u1','item_id':'b3'},
        {'user_id':'u2','item_id':'b2'},
        {'user_id':'u2','item_id':'b5'},
        {'user_id':'u3','item_id':'b4'},
    ])

    # normalize genres
    items['genres'] = items['genres'].apply(ensure_genres_list)

    train, test = train_test_split_leave_one_out(interactions)

    user2idx, idx2user, item2idx, idx2item = build_mappings(train, items)
    train_mat = build_user_item_matrix(train, user2idx, item2idx)

    # instantiate models
    pop = PopularityRecommender()
    svd = SVDRecommender(n_factors=2)
    graph = GraphRecommender()

    models = {'popularity': pop, 'svd': svd, 'graph': graph}

    res = evaluate_models(train, test, items, models, k_list=[1,3])
    print(res)

Fitting popularity
Fitting svd
Fitting graph
Evaluating popularity
Evaluating svd
Evaluating graph
        model  k  precision  recall      ndcg
0  popularity  1   0.000000     0.0  0.000000
1  popularity  3   0.000000     0.0  0.000000
2         svd  1   0.000000     0.0  0.000000
3         svd  3   0.333333     1.0  0.565465
4       graph  1   0.000000     0.0  0.000000
5       graph  3   0.000000     0.0  0.000000
